In [1]:
import pandas as pd

fitches = pd.read_csv("final_langs.csv")

In [ ]:
fitches

In [61]:
fitches.dropna(inplace = True)
fitches.drop(columns = ['fitcha'], inplace = True)

In [34]:
from translate import Translator
translator= Translator(to_lang="Russian")

fitches['title'] = fitches['title'].apply(lambda x: translator.translate(x))

RuntimeError: generator raised StopIteration

In [62]:
fitches

,Unnamed: 0,title,description,rating,subs,specification
2,2,DevOps,Software Development Methodology,206.29,1200.0,method
29,29,PowerShell,Task automation and configuration management f...,1.80,8500.0,framework
31,31,Serverless,Serverless computing technologies,0.50,9600.0,technolog
35,35,Fidonet,Worldwide amateur computer network,0.00,7800.0,ide
49,49,Working with video,All about creating and processing videos,8.44,15100.0,ide
...,...,...,...,...,...,...
413,413,CakePHP,A framework for creating web-applications,0.00,13700.0,framework
419,419,Perl,"Perl is a high-level, interpreted, dynamic pro...",-0.80,14800.0,language
420,420,Google App Engine,A web framework and cloud computing platform f...,-1.00,8300.0,framework
421,421,Solidity,Solidity is a contract-oriented programming la...,-1.53,1900.0,language


# Работа с роспатентом

In [2]:
import requests
import json

texts = []

def get_params_by_fitcha(query):
    try:
        raw = requests.post('https://searchplatform.rospatent.gov.ru/search?t=1659017631255', 
                    json = {"qn":query,
                            "offset":0,"limit":1000,
                            "include_facets":1,
                            "filter":{"country":{"values":["RU"]}},"sort":"relevance","highlight":{"profiles":["_searchquery_"]},"datasets":["ru_till_1994","ru_since_1994","cis","ap","cn","ch","au","gb","kr","ca","at","jp","ep","de","fr","pct","us","others"],"preffered_lang":"ru"})
        fs = raw.json()
        print(query + " articles is ready\n_____________________________________")
    
        descriptions = []
        classes = []

        for i in fs['hits']:
            descriptions.append(i['snippet']['description'])
            classes.append(i['snippet']['classification']['ipc'])

        return descriptions
    except:
        return ["No data"]

fitcha_with_articles = pd.DataFrame()

for fitcha in set(fitches['language'].to_numpy()):
    descriptions = get_params_by_fitcha(fitcha)
    temp_df = pd.DataFrame({"name": fitcha, "article": descriptions})

    fitcha_with_articles = pd.concat([fitcha_with_articles, temp_df])

Vala articles is ready
_____________________________________
Smalltalk articles is ready
_____________________________________
BASIC articles is ready
_____________________________________
Groovy articles is ready
_____________________________________
Swift articles is ready
_____________________________________
Go articles is ready
_____________________________________
Python articles is ready
_____________________________________
Self articles is ready
_____________________________________
Java articles is ready
_____________________________________
C++ articles is ready
_____________________________________
Ruby articles is ready
_____________________________________
Dart articles is ready
_____________________________________
C# articles is ready
_____________________________________
ColdFusion articles is ready
_____________________________________
Ada articles is ready
_____________________________________
Eiffel articles is ready
_____________________________________
Io articles

In [3]:
fitcha_with_articles.to_csv("rospatent_by_lang.csv", index = False)

In [5]:
fitcha_with_articles.groupby('Specification').size().plot(kind='bar', figsize = (30, 30))

KeyError: 'Specification'

In [124]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from string import punctuation
russian_stopwords = stopwords.words("russian")

def remove_punct(text):
    table = {33: ' ', 34: ' ', 35: ' ', 36: ' ', 37: ' ', 38: ' ', 39: ' ', 40: ' ', 41: ' ', 42: ' ', 43: ' ', 44: ' ', 45: ' ', 46: ' ', 47: ' ', 58: ' ', 59: ' ', 60: ' ', 61: ' ', 62: ' ', 63: ' ', 64: ' ', 91: ' ', 92: ' ', 93: ' ', 94: ' ', 95: ' ', 96: ' ', 123: ' ', 124: ' ', 125: ' ', 126: ' '}
    return text.translate(table)

fitcha_with_articles['article_clean'] = fitcha_with_articles['article'].map(lambda x: x.lower())
fitcha_with_articles['article_clean'] = fitcha_with_articles['article_clean'].map(lambda x: remove_punct(x))
fitcha_with_articles['article_clean'] = fitcha_with_articles['article_clean'].map(lambda x: x.replace('em', ''))
fitcha_with_articles['article_clean'] = fitcha_with_articles['article_clean'].map(lambda x: x.split(' '))
fitcha_with_articles['article_clean'] = fitcha_with_articles['article_clean'].map(lambda x: [token for token in x if token not in russian_stopwords\
                                                                  and token != " " \
                                                                  and token.strip() not in punctuation])
fitcha_with_articles['article_clean'] = fitcha_with_articles['article_clean'].map(lambda x: ' '.join(x))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kingo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [125]:
fitcha_with_articles

,name,description,rating,subs,article,ros_pt,Specification,article_clean
0,DevOps,Software Development Methodology,206.29,1200.0,URL: https:&#x2F;&#x2F;devblogs.microsoft.com&...,H04L12/00,method,url https x2f x2f devblogs microsoft com x2f d...
1,DevOps,Software Development Methodology,206.29,1200.0,"В частности, за счет обеспечения автоматизиров...",G06F21/57,method,частности счет обеспечения автоматизированной ...
2,DevOps,Software Development Methodology,206.29,1200.0,В качестве примера этапы процесса обновления в...,G06F8/76,method,качестве примера этапы процесса обновления авт...
0,PowerShell,Task automation and configuration management f...,1.80,8500.0,В одном из вариантов реализации изобретения до...,G06F21/55,framework,одном вариантов реализации изобретения доверен...
1,PowerShell,Task automation and configuration management f...,1.80,8500.0,"Например, современные направленные атаки могут...",G06F21/55,framework,например современные направленные атаки могут ...
...,...,...,...,...,...,...,...,...
995,XML,Extensible Markup Language,-3.67,8500.0,Дополнительно в большинстве случаев подробност...,G06Q30/00,language,дополнительно большинстве случаев подробности ...
996,XML,Extensible Markup Language,-3.67,8500.0,В примерном варианте осуществления CGDM имеет ...,G06F8/36,language,примерном варианте осуществления cgdm имеет i ...
997,XML,Extensible Markup Language,-3.67,8500.0,"Эти параметры могут сохраняться в файле, имеющ...",G16H40/63,language,параметры могут сохраняться файле имеющем форм...
998,XML,Extensible Markup Language,-3.67,8500.0,База данных когнитивных моделей может состоять...,G06F17/30,language,база данных когнитивных моделей состоять неско...


In [126]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(fitcha_with_articles['article_clean'], fitcha_with_articles['Specification'], test_size=0.1, random_state=42)


In [127]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV


In [128]:
sgd_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42))])
knb_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('knb_clf', KNeighborsClassifier(n_neighbors=15))])
sgd_ppl_clf.fit(X_train, y_train)
knb_ppl_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('knb_clf', KNeighborsClassifier(n_neighbors=15))])

In [129]:
predicted_sgd = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))


              precision    recall  f1-score   support

   algorithm       0.68      0.96      0.80        80
   framework       0.82      0.97      0.89       438
         ide       0.83      0.92      0.87       520
    language       0.98      0.89      0.93      2155
     library       0.00      0.00      0.00         0
      method       0.74      0.96      0.83        90
   technolog       0.89      0.97      0.93       180

    accuracy                           0.91      3463
   macro avg       0.71      0.81      0.75      3463
weighted avg       0.92      0.91      0.91      3463



c:\users\kingo\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\kingo\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\kingo\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [134]:
inputs = input()
predicted_sgd = sgd_ppl_clf.predict([inputs])

predicted_sgd

Устройство реализации коптера


array(['language'], dtype='<U9')

In [135]:
Нужно решение, чтобы реализовать парсинг

SyntaxError: invalid syntax (533679477.py, line 1)

In [137]:
check_leninka = pd.read_csv('Leninka_fitches_big_categories.csv')
check_leninka

,Unnamed: 0,ID,Name,Annotations,Authors,Years,Links,Journals,Journal_links,Catalogs,Сфера ИТ
0,0,devops-concepts,<b>DevOps</b> concepts,In this article the author discusses the conce...,['Makosi Roman'],2017,/article/n/devops-concepts,Вестник Хакасского государственного университе...,/journal/n/vestnik-hakasskogo-gosudarstvennogo...,NaN,DevOps
1,1,uskorenie-nepreryvnoy-integratsii-i-razvertyva...,УСКОРЕНИЕ НЕПРЕРЫВНОЙ ИНТЕГРАЦИИ И РАЗВЕРТЫВАН...,в работе описана концепция непрерывной интегра...,['Шляпников Валерий Михайлович'],2021,/article/n/uskorenie-nepreryvnoy-integratsii-i...,Инновационные аспекты развития науки и техники,/journal/n/innovatsionnye-aspekty-razvitiya-na...,NaN,DevOps
2,2,sravnitelnyy-analiz-informatsionnyh-sistem-men...,СРАВНИТЕЛЬНЫЙ АНАЛИЗ ИНФОРМАЦИОННЫХ СИСТЕМ МЕН...,Проведен сравнительный анализ трех информацион...,"['Короходкин Д.В.', 'Гагарина С.Н.']",2021,/article/n/sravnitelnyy-analiz-informatsionnyh...,Экономика и бизнес: теория и практика,/journal/n/ekonomika-i-biznes-teoriya-i-praktika,NaN,DevOps
3,3,postroenie-i-soprovozhdenie-proektov-so-slozhn...,Построение и сопровождение проектов со сложной...,"Статья посвящена рассмотрению проблем, возника...","['Таборовец Вячеслав Васильевич', 'Былина Алек...",2018,/article/n/postroenie-i-soprovozhdenie-proekto...,Проблемы современной науки и образования,/journal/n/problemy-sovremennoy-nauki-i-obrazo...,NaN,DevOps
4,4,structural-analysis-and-optimization-of-idef0-...,Structural analysis and optimization of IDEF0 ...,Recommendations obtained as a result of IDEF0 ...,"['Godlevskyi M. D.', 'Orlovskyi D. L.', 'Kopp ...",2018,/article/n/structural-analysis-and-optimizatio...,"Радіоелектроніка, інформатика, управління",/journal/n/radioelektronika-informatika-upravl...,"[{'id': 1, 'alias_name': 'wos', 'acronym': 'WO...",DevOps
...,...,...,...,...,...,...,...,...,...,...,...
37233,37233,on-paradigm-shift-in-telecommunication-technol...,ON PARADIGM SHIFT IN TELECOMMUNICATION TECHNOL...,The article is devoted to the discussion of th...,"['Sneps-Sneppe Manfred', 'Namiot Dmitry']",2021,/article/n/on-paradigm-shift-in-telecommunicat...,International Journal of Open Information Tech...,/journal/n/international-journal-of-open-infor...,"[{'id': 8, 'alias_name': 'vak', 'acronym': 'ВА...",XML
37234,37234,dialogovaya-sistema-dream-v-konkurse-alexa-pri...,ДИАЛОГОВАЯ СИСТЕМА DREAM В КОНКУРСЕ ALEXA PRIZ...,"Создание диалоговой системы, способной быстро,...","['Куратов Ю. М.', 'Юсупов И. Ф.', 'Баймурзина ...",2021,/article/n/dialogovaya-sistema-dream-v-konkurs...,Труды Московского физико-технического института,/journal/n/trudy-moskovskogo-fiziko-tehnichesk...,"[{'id': 8, 'alias_name': 'vak', 'acronym': 'ВА...",XML
37235,37235,gis-segodnya-sostoyanie-perspektivy-resheniya,"ГИС сегодня: состояние, перспективы, решения",Дан краткий анализ современного состояния геои...,"['Шокин Юрий Иванович', 'Потапов Вадим Петрович']",2015,/article/n/gis-segodnya-sostoyanie-perspektivy...,Вычислительные технологии,/journal/n/vychislitelnye-tehnologii,"[{'id': 22, 'alias_name': 'rsci', 'acronym': '...",XML
37236,37236,analiz-modeley-i-metodik-ispolzuemyh-dlya-atri...,"АНАЛИЗ МОДЕЛЕЙ И МЕТОДИК, ИСПОЛЬЗУЕМЫХ ДЛЯ АТР...","Цель работы: анализ моделей и методик, использ...","['Котенко Игорь', 'Хмыров Семен Сергеевич']",2022,/article/n/analiz-modeley-i-metodik-ispolzuemy...,Вопросы кибербезопасности,/journal/n/voprosy-kiberbezopasnosti,"[{'id': 8, 'alias_name': 'vak', 'acronym': 'ВА...",XML
